In [22]:
import numpy as np 
import pandas as pd
import os

In [23]:
frieght = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name='FreightRates')
order = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name='OrderList')
plant_port = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name='PlantPorts')
prod_per_plant = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name='ProductsPerPlant')
vim_cust = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name='VmiCustomers')
wh_cap= pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name='WhCapacities')
wh_cost = pd.read_excel('Supply chain logisitcs problem.xlsx', sheet_name='WhCosts')

In [24]:
frieght

,Carrier,orig_port_cd,dest_port_cd,minm_wgh_qty,max_wgh_qty,svc_cd,minimum cost,rate,mode_dsc,tpt_day_cnt,Carrier type
0,V444_8,PORT02,PORT09,100.0,249.99,DTD,17.5156,0.0484,AIR,14,V88888888_0
1,V444_8,PORT02,PORT09,0.0,99.99,DTD,17.5156,0.0484,AIR,14,V88888888_0
2,V444_8,PORT02,PORT09,250.0,499.99,DTD,17.5156,0.0484,AIR,14,V88888888_0
3,V444_8,PORT02,PORT09,2000.0,99999.99,DTD,17.5156,0.0484,AIR,14,V88888888_0
4,V444_8,PORT02,PORT09,500.0,1999.99,DTD,17.5156,0.0484,AIR,14,V88888888_0
...,...,...,...,...,...,...,...,...,...,...,...
1535,V444_8,PORT11,PORT09,100.0,249.99,DTD,27.7884,0.2028,AIR,0,V88888888_0
1536,V444_8,PORT11,PORT09,2000.0,99999.99,DTD,27.7884,0.2028,AIR,0,V88888888_0
1537,V444_8,PORT11,PORT09,250.0,499.99,DTD,27.7884,0.2028,AIR,0,V88888888_0
1538,V444_8,PORT11,PORT09,0.0,99.99,DTD,27.7884,0.3448,AIR,0,V88888888_0


In [25]:
dict(zip(frieght['orig_port_cd'], frieght['dest_port_cd']))

{'PORT02': 'PORT09',
 'PORT03': 'PORT09',
 'PORT04': 'PORT09',
 'PORT05': 'PORT09',
 'PORT06': 'PORT09',
 'PORT07': 'PORT09',
 'PORT08': 'PORT09',
 'PORT09': 'PORT09',
 'PORT10': 'PORT09',
 'PORT11': 'PORT09'}

In [26]:
order.isnull().sum()

Order ID                0
Order Date              0
Origin Port             0
Carrier                 0
TPT                     0
Service Level           0
Ship ahead day count    0
Ship Late Day count     0
Customer                0
Product ID              0
Plant Code              0
Destination Port        0
Unit quantity           0
Weight                  0
dtype: int64

In [27]:
plant_port.isnull().sum()

Plant Code    0
Port          0
dtype: int64

In [28]:
prod_per_plant.isnull().sum()

Plant Code    0
Product ID    0
dtype: int64

In [29]:
wh_cap.isnull().sum()

Plant ID           0
Daily Capacity     0
dtype: int64

In [30]:
wh_cost.isnull().sum()

WH           0
Cost/unit    0
dtype: int64

In [31]:
vim_cust.isnull().sum()

Plant Code    0
Customers     0
dtype: int64

In [32]:
order = order[['Order Date', 'Order ID', 'Carrier', 'Origin Port', 'Destination Port', 'Unit quantity', 'Weight']]
df_merge = pd.merge(order, plant_port, left_on=['Origin Port'], right_on=['Port'])
df_merge = df_merge.rename(columns= {'Planr Code' : 'origin_plant_code'})
df_merge = df_merge.drop(['Port'], axis=1)
df_merge = pd.merge(df_merge, plant_port, left_on=['Destination Port'], right_on=['Port'])
df_merge = df_merge.drop(['Port'], axis=1)
df_merge = df_merge.rename(columns= {'Plant Code' : 'destination_plant_code'})
df_merge.head()

,Order Date,Order ID,Carrier,Origin Port,Destination Port,Unit quantity,Weight,Plant Code_x,Plant Code_y
0,2013-05-26,1.447307e+09,V44_3,PORT04,PORT09,1529,14.3,PLANT03,PLANT16
1,2013-05-26,1.447307e+09,V44_3,PORT04,PORT09,1529,14.3,PLANT08,PLANT16
2,2013-05-26,1.447307e+09,V44_3,PORT04,PORT09,1529,14.3,PLANT09,PLANT16
3,2013-05-26,1.447307e+09,V44_3,PORT04,PORT09,1529,14.3,PLANT11,PLANT16
4,2013-05-26,1.447307e+09,V44_3,PORT04,PORT09,1529,14.3,PLANT12,PLANT16


In [33]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63461 entries, 0 to 63460
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Order Date        63461 non-null  datetime64[ns]
 1   Order ID          63461 non-null  float64       
 2   Carrier           63461 non-null  object        
 3   Origin Port       63461 non-null  object        
 4   Destination Port  63461 non-null  object        
 5   Unit quantity     63461 non-null  int64         
 6   Weight            63461 non-null  float64       
 7   Plant Code_x      63461 non-null  object        
 8   Plant Code_y      63461 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 4.8+ MB


In [34]:
freight_new = frieght.groupby(['Carrier', 'orig_port_cd','dest_port_cd']).agg({
    'minm_wgh_qty' : lambda x : sum(x)/len(x),
    'max_wgh_qty' : lambda x : sum(x)/len(x),
    'rate' : lambda x : sum(x)/len(x),
    'minimum cost' : lambda x : sum(x)/len(x),
}).reset_index().rename(columns = {'minimum cost': 'min_cost', 'minm_wgh_qty' : 'min_wgh_qty'})

freight_new

,Carrier,orig_port_cd,dest_port_cd,min_wgh_qty,max_wgh_qty,rate,min_cost
0,V444_0,PORT02,PORT09,570.000000,20569.990000,0.084480,23.838400
1,V444_0,PORT03,PORT09,0.000000,5000.000000,12.694200,28.944200
2,V444_0,PORT04,PORT09,570.000000,20569.990000,0.052000,2.477200
3,V444_0,PORT09,PORT09,0.000000,5000.000000,12.778400,31.278400
4,V444_1,PORT02,PORT09,197.556000,1190.745000,0.291140,23.557340
5,V444_1,PORT04,PORT09,197.556000,1190.745000,0.242000,20.909780
6,V444_1,PORT05,PORT09,170.130000,1001.952500,0.457895,35.954091
7,V444_1,PORT06,PORT09,45.473493,111.272697,2.492247,6.917300
8,V444_1,PORT10,PORT09,78.945000,12570.433750,0.431400,12.350850
9,V444_2,PORT02,PORT09,570.000000,20569.990000,0.079040,9.027200


In [35]:
cost = freight_new.groupby(['orig_port_cd','dest_port_cd'])['min_cost'].min().unstack()
cost = round(cost,0)
cost = cost[cost.index.isin(df_merge['Origin Port'])]
cost_pulp = cost.to_numpy().tolist()
cost_pulp

[[2.0], [8.0], [24.0]]

In [36]:
cost

dest_port_cd,PORT09
orig_port_cd,
PORT04,2.0
PORT05,8.0
PORT09,24.0


In [37]:
wh_cap

,Plant ID,Daily Capacity
0,PLANT15,11
1,PLANT17,8
2,PLANT18,111
3,PLANT05,385
4,PLANT02,138
5,PLANT01,1070
6,PLANT06,49
7,PLANT10,118
8,PLANT07,265
9,PLANT14,549


In [38]:
origins = df_merge['Origin Port'].unique().tolist()
origins = sorted(origins)
destinations = df_merge['Destination Port'].unique().tolist()

df_origin = df_merge.groupby(['Origin Port']).agg({
    'Order ID' : 'nunique',
    'Order Date' : 'nunique',
    'Unit quantity' : 'sum',
    'Weight' : 'sum'
}).reset_index()

df_origin['avg_qty'] = round(df_origin['Unit quantity']/df_origin['Order Date'],0)
df_origin['avg_weight'] = round(df_origin['Weight']/df_origin['Order Date'],0)
df_origin = df_origin.drop(['Order ID','Unit quantity','Weight'], axis = 1)


df_dest = df_merge.groupby(['Destination Port']).agg({
    'Order ID' : 'nunique',
    'Order Date' : 'nunique',
    'Unit quantity' : 'sum',
    'Weight' : 'sum'
}).reset_index()

df_dest['avg_qty'] = round(df_dest['Unit quantity']/df_dest['Order Date'],0)
df_dest['avg_weight'] = round(df_dest['Weight']/df_dest['Order Date'],0)
df_dest = df_dest.drop(['Order ID','Unit quantity','Weight'], axis = 1)

print(df_dest)
print(df_origin)

  Destination Port  Order Date      avg_qty  avg_weight
0           PORT09           1  205120319.0   1277181.0
  Origin Port  Order Date      avg_qty  avg_weight
0      PORT04           1  204874838.0   1276407.0
1      PORT05           1        348.0         7.0
2      PORT09           1     245133.0       767.0


In [39]:
supply = dict(zip(df_origin['Origin Port'], df_origin['avg_weight']))
demand = dict(zip(df_dest['Destination Port'], df_dest['avg_weight']))
print(f'All Origin Nodes : {origins}')
print(f'All Origin Supply : {supply}')
print(f'All Destination Nodes : {destinations}')
print(f'All Destination Demand : {demand}')
print(f'cost from Origin to Destination: {cost_pulp}')

All Origin Nodes : ['PORT04', 'PORT05', 'PORT09']
All Origin Supply : {'PORT04': 1276407.0, 'PORT05': 7.0, 'PORT09': 767.0}
All Destination Nodes : ['PORT09']
All Destination Demand : {'PORT09': 1277181.0}
cost from Origin to Destination: [[2.0], [8.0], [24.0]]


In [40]:
### cost for transport from each nodes

from pulp import *

costs = makeDict([origins,destinations], cost_pulp, 0)
prob = LpProblem()

routes = [(o,d) for o in origins for d in destinations]

vars = LpVariable.dicts('route', (origins,destinations), 0, None, LpInteger)

prob += (lpSum([vars[o][d] * costs[o][d] for (o,d) in routes]), 'sum of Transportation costs')

prob

NoName:
MINIMIZE
2.0*route_PORT04_PORT09 + 8.0*route_PORT05_PORT09 + 24.0*route_PORT09_PORT09 + 0.0
VARIABLES
0 <= route_PORT04_PORT09 Integer
0 <= route_PORT05_PORT09 Integer
0 <= route_PORT09_PORT09 Integer

In [41]:
### supply max constrains from each nodes


for o in origins :
    prob += (lpSum([vars[o][d] for d in destinations]) <= supply[o], 'sum_of_weights_of_goods_into_origin_ports_%s' %o,)
for d in destinations:
    prob += (lpSum([vars[o][d] for o in origins]) >= demand[d], 'sum_of_weights_of_goods_into_destination_ports_%s' %d,)

In [42]:
prob.solve()
for v in prob.variables():
    print(f'{v} : {v.varValue}')

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/sangru/anaconda3/lib/python3.7/site-packages/pulp/apis/../solverdir/cbc/linux/64/cbc /tmp/b9e51bb8ed234254ad63d1d2cdee20ee-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/b9e51bb8ed234254ad63d1d2cdee20ee-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 25 RHS
At line 30 BOUNDS
At line 34 ENDATA
Problem MODEL has 4 rows, 3 columns and 6 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 2.57128e+06 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 2.57128e+06 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times